In [1]:
df = sqlContext.read.load("pawn.txt", 
                      format='com.databricks.spark.csv', 
                      header='true', delimiter='\t',mode='PERMISSIVE',
                     inferSchema='true')

IllegalArgumentException: 'Delimiter cannot be more than one character: \t+'

In [2]:
pawn=df.rdd.map(lambda x:x.Description)

In [3]:
pawn_all=df.rdd.map(lambda x:x).zipWithIndex().toDF()

In [4]:
pawn_doc=df.rdd.map(lambda x:x.Description).zipWithIndex().toDF()

In [5]:
pawn_doc.show()

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|1 Bracelet   yell...|  0|
|1 Earring  14 kt ...|  1|
|1 Earring  14 kt ...|  2|
|1 Chain   yellow ...|  3|
|1 Charm   yellow ...|  4|
|14 kt yellow gold...|  5|
|           Demo     |  6|
+--------------------+---+



In [36]:
import re
#from stemming.porter2 import stem
def clean_word(w):
    w=w.lower().strip()
    w=re.sub('\n'," ",w)
    #w=re.sub(r'\s+', ' ', w).strip()
    w=re.sub("[^a-z| |0-9]|,\,|\.|\;|\:|\;|\?|\!|\[|\]|\}|\{(?i)\b|[0-9]|((?:https?://|www\d{0,3}))|[//]|[#]|[$]|", "", (w.lower()))
    w=re.sub('\s\s+', ' ', w)
    return w.lower().strip()
 

In [37]:
text=pawn_doc.rdd.map(lambda x: (clean_word(x[0]),x[1])).toDF().withColumnRenamed("_1","doc_text").withColumnRenamed("_2","doc_id")

In [38]:
df = (text
      .rdd
  .map(lambda x : (x.doc_id, x.doc_text.split(" ")))
  .toDF()
  .withColumnRenamed("_1","doc_id")
  .withColumnRenamed("_2","features"))

In [39]:
#df.withcolum("feature",filter())

In [40]:
#kuchAur.show()

In [47]:
df.count()

7

In [42]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="features", outputCol="tags")
df0=remover.transform(df)
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF
from pyspark.sql.types import DoubleType
htf = HashingTF(inputCol="tags", outputCol="tf")
tf = htf.transform(df0)
tf1=tf.select("doc_id","tags")


In [43]:
pawn_rest=pawn_all.rdd.map(lambda x: (x[0],x[1])).toDF().withColumnRenamed("_1","Pawn").withColumnRenamed("_2","doc_id")

In [44]:
in_df=pawn_rest.join(tf1,["doc_id"]).drop("doc_id")

{"Pawn":{"StoreName":"Hope Diamonds","StoreAddress":"275 route 4 West, #12","StoreCity":"Paramus","StoreState":"NJ","StoreZip":7652,"StorePhone":"(201)489-8898","TransDate":"28/02/2017 18:02","Amount":259,"LastName":"kartanos","FirstName":"steve","MiddleName":"NULL","DBO":"08/04/1982 0:00","Gender":"NULL","Address":"384 LACEY DR","City":"NEW MILFORD","State":"NJ","ZIP":7646,"HomePhone":"NULL","WorkPhone":"NULL","CellPhone":"NULL","Description":"1 Bracelet   yellow  5.2 grams","Make":"NULL","Mode":"NULL","Color":"yellow","Serial":"NULL","ItemAmount":98},"tags":["bracelet","yellow","grams"]}


In [46]:
in_df.coalesce(1).write.format('json').save('pawn.json')

AttributeError: 'str' object has no attribute 'tags'

In [ ]:
df = (text
      .rdd
  .map(lambda x : (x.doc_id,x.doc_text.split(" ")))
  .toDF()
  .withColumnRenamed("_1","doc_id")
  .withColumnRenamed("_2","features"))
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="features", outputCol="tags")
df0=remover.transform(df)
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF
from pyspark.sql.types import DoubleType
htf = HashingTF(inputCol="tags", outputCol="tf")
tf = htf.transform(df0)
tf1=tf.select("doc_id","tags")


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA

In [ ]:
import re
#from stemming.porter2 import stem
def clean_word(w):
   # w=w.lower().strip()
    w=re.sub('\n'," ",w)
    return re.sub("[^a-z| |0-9]|,\,|\.|\;|\:|\;|\?|\!|\[|\]|\}|\{(?i)\b|[0-9]|((?:https?://|www\d{0,3}))|[//]|[#]|[$]|", "", (w.lower()))


In [ ]:
pawn_clean=pawn.map(lambda x: clean_word(x))

In [ ]:
pawn1= pawn_clean.flatMap(lambda x: x.split(",")).collect()

In [ ]:
#def tfidf (text):
vect = CountVectorizer(ngram_range=(1,2),max_df=0.7 ) #stop_words='None'
vocabulary =  vect.fit(pawn1).vocabulary_
tfidf_vect = TfidfVectorizer(vocabulary=vocabulary,ngram_range=(1,2),max_df=0.7) #stop_words='None'

In [ ]:
feat_pawn=tfidf_vect.fit_transform(pawn1)

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,LongType
#schema = StructType([StructField("k", StringType(), True), StructField("v", IntegerType(), False)])
#schema = StructType([StructField("postId", LongType(), True),StructField("tags", StringType(), True)])
schema = StructType([StructField("tags", StringType(), True)])
#schema = StructType(("tags", StringType(), True))
#schema = StructType([])
empty_idn = sqlContext.createDataFrame(sc.emptyRDD(), schema)
schema1 = StructType([StructField("postId", StringType(), True)])
#schema1 = StructType([])
empty_idn1 = sqlContext.createDataFrame(sc.emptyRDD(), schema1)

In [ ]:
for i in range (3):
    idn=pawn_doc.select(pawn_doc._2).filter (pawn_doc._2 == i)
    #idn=pawn_doc.filter(lambda x: x[1]== i).map (lambda x:x[1]).zipWithIndex().toDF()
    #idn1=idn.select("_1")
    #idn1=idn.rdd.zipWithIndex().toDF().withColumnRenamed('_1', 'tags').withColumnRenamed('_2', 'index')
    feature_array = np.array(tfidf_vect.get_feature_names())
    tfidf_sorting = (feat_pawn[i].nonzero()[1])
    top_n = feature_array[tfidf_sorting]
    top=sc.parallelize([top_n],1).map(lambda x: str(x)).zipWithIndex().toDF()
    top1=top.select("_1")
    #rdd = (idn.rdd.zip(top)) 
    #daf=sqlContext.createDataFrame(top)
    #daf=idn1.join(top,["index"]).drop("index")
    empty_idn=empty_idn.union(top1)
    empty_idn1=empty_idn1.union(idn)

In [ ]:
empty_id=empty_idn.rdd.zipWithIndex().toDF().withColumnRenamed('_1', 'tags').withColumnRenamed('_2', 'index')
empty_id1=empty_idn1.rdd.zipWithIndex().toDF().withColumnRenamed('_1', 'postId').withColumnRenamed('_2', 'index')#f

In [ ]:
in_df=empty_id1.join(empty_id,["index"]).drop("index")

In [ ]:
indf1=in_df.rdd.map(lambda l: (l[0],l[1]))

In [ ]:
indf1.coalesce(1).write.format('json').save('test1.json') ###write
indf1.coalesce(1).format('json').save('test1.json') ###write

In [ ]:
import json
multiline_rdd=sc.wholeTextFiles('SMALL.JSON')
json_rdd = multiline_rdd.map(lambda x : x[1])
from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)
#sqlContext=SQLContext(sc)
d1=sqlContext.read.json(json_rdd)
d3=d1.dropDuplicates()

In [ ]:
print (d3.printSchema())

In [ ]:
coll=d3.rdd.map(lambda x: (x.description,x.loc.coordinates))

In [ ]:
collab=d3.rdd.map(lambda x:x.description)

In [ ]:
collab_doc=d3.rdd.map(lambda x:(x.description,x.loc.coordinates)).zipWithIndex()

In [ ]:
collab_doc.take(1)

In [ ]:
collab_clean=collab.map(lambda x: clean_word(x))
collab1= collab_clean.flatMap(lambda x: x.split(",")).collect()

In [ ]:
#def tfidf (text):
vect = CountVectorizer(stop_words='english')
vocabulary =  vect.fit(collab1).vocabulary_
tfidf_vect = TfidfVectorizer(stop_words='english',vocabulary=vocabulary)
    #return tfidf_vect.fit_transform(text)

In [ ]:
feat=vect.transform(collab1)

In [ ]:
feature_names = vect.get_feature_names()
list1=[]
for i in range(len(collab1)):
    for col in feat[i].nonzero()[1]:
        if feature_names[col]== 'burglary':
            list1.append(i)

In [ ]:
collab2=collab_doc.filter(lambda x: x[1]==x[1] in list1).map (lambda x: (x[0]))

In [ ]:
collab2.take(1)

In [ ]:
df=collab2.toDF().withColumnRenamed("_1","doc_text").withColumnRenamed("_2","coord")

In [ ]:
zipc

In [ ]:
def clean_zip(text):
    text= str (text)
    text=text.replace('<Zip:'  ,"")
    text=text.replace('>',"")
    text=text.replace('[',"")
    text=text.replace (']',"")
    text= [x.strip() for x in text.split(',')]
    return [int(float(a)) for a in text]
    #return text

In [ ]:
clean_zip(zipc)

In [ ]:
zipc7=[int(float(a)) for a in zipc6]

In [ ]:
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
import re
def clean_coord(w):
    w= str(w)
    #w=re.sub('[\\]',"",w)
    #w=w[:10]
    w=str(w).replace('[','').replace(']','')
    return w
    #
udf_clean_coord = udf(clean_coord, StringType())
df1=df.withColumn("coordinates", udf_clean_coord("coord"))

In [1]:
import CMUTweetTagger

ImportError: No module named 'CMUTweetTagger'

In [ ]:
df2.show()

In [ ]:
split_col = split(df1['coordinates'], ',')
df1 = df1.withColumn('latitude',  (split_col.getItem(0)))#.drop("coord")
df2= df1.withColumn('longitude', (split_col.getItem(1))) #.drop ("coord")
df2=df2.withColumn ('lat',df2.latitude.cast('float'))
df2=df2.withColumn ('long',df2.longitude.cast('float'))
#rawdata.price.cast

In [ ]:
def zipcode1(x, y, z=5):
    zipc=(zipcode.isinradius((x,y),z))
    return zipc

In [ ]:
from pyspark.sql.types import *
#udf_zipcode = udf(zipcode1,ArrayType(StringType()))
udf_zipcode = udf()

In [ ]:
rdd_zipcode=df2.rdd.map(lambda x: zipcode1(x.long,x.lat))

In [ ]:
rdd_zipcode.take(2)

In [ ]:
#zipc=zipcode.isinradius(cbus, 20)
df3=df2.withColumn ('zipcode',udf_zipcode(df2.long,df2.lat))

In [ ]:
df3.show()

In [ ]:
df2.select('zipcode').show()

In [ ]:
collab_burg1=collab2.zipWithIndex()

In [ ]:
collab_clean1=collab2.map(lambda x: clean_word(x))
collab3= collab_clean1.flatMap(lambda x: x.split(",")).collect()

In [ ]:
#def tfidf (text):
vect1 = CountVectorizer(stop_words='english')
vocabulary1 =  vect1.fit(pawn1+collab3).vocabulary_
tfidf_vect1 = TfidfVectorizer(stop_words='english',vocabulary=vocabulary)
    #return tfidf_vect.fit_transform(text)
                             

In [ ]:
pawn_mat=tfidf_vect1.fit_transform(pawn1)

In [ ]:
collab1_mat=tfidf_vect1.fit_transform(collab3)

In [ ]:
def broadcast_matrix(mat):
    bcast = sc.broadcast((mat.data, mat.indices, mat.indptr))
    (data, indices, indptr) = bcast.value
    bcast_mat = csr_matrix((data, indices, indptr), shape=mat.shape)
    return bcast_mat

def parallelize_matrix(scipy_mat, rows_per_chunk=10):
    [rows, cols] = scipy_mat.shape
    i = 0
    submatrices = []
    while i < rows:
        current_chunk_size = min(rows_per_chunk, rows - i)
        submat = scipy_mat[i:i + current_chunk_size]
        submatrices.append((i, (submat.data, submat.indices, 
                                submat.indptr),
                            (current_chunk_size, cols)))
        i += current_chunk_size
    return sc.parallelize(submatrices)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
a_mat_para = parallelize_matrix(collab1_mat, rows_per_chunk=100)
b_mat_dist = broadcast_matrix(pawn_mat)
from sklearn.metrics.pairwise import cosine_similarity

def find_matches_in_submatrix(sources, targets, inputs_start_index,
                              threshold=.1):
    #count = 0
    cosimilarities = cosine_similarity(sources, targets)
    for i, cosimilarity in enumerate(cosimilarities):
        cosimilarity = cosimilarity.flatten()
        index=np.argsort(-cosimilarity).tolist()
        target_index = index#
        #target_index = cosimilarity.argsort()[-1]
        #target_index = -cosimilarity.argsort().tolist()
        source_index = inputs_start_index + i
        for j in range(3):
        #for target_index in enumerate(target_index):
            target_index =index[j]
            similarity = cosimilarity[target_index]
            if cosimilarity[target_index] > threshold:
                #k=k+j
                #count += 1
                yield (source_index, target_index,similarity)

In [ ]:
x= (a_mat_para.flatMap(lambda submatrix:find_matches_in_submatrix(csr_matrix(submatrix[1],shape=submatrix[2]),b_mat_dist,submatrix[0])))

In [ ]:
x.take(15)

In [ ]:
y1=x.map (lambda x: x[0]).collect()
y2=x.map (lambda x: x[1]).collect()

In [ ]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
y=x.map(lambda x: (x[0],x[1],x[2]))
#y1=(y.zipWithIndex().toDF().withColumnRenamed("_1","DOC").withColumnRenamed("_2","Related_doc").withColumnRenamed("_3","id"))

In [ ]:
for i in range(6):##### change here
    df4=collab_burg.filter(lambda x: x[1]==y1[i]).map (lambda x: x[0]).collect()
    print (df4)
    print ('='*60)
               
    df5=pawn_doc.filter(lambda x: x[1]==(y2[i])).map (lambda x: x[0]).collect()
        
    print (df5)
    print ('-'*100 )
    

In [ ]:
feature_names = tfidf_vect1.get_feature_names()
for i in range (5): 
    a=y2[i]
    b=y1[i]
    for col in pawn_mat[a].nonzero()[1]:
        for col1 in collab1_mat[b].nonzero()[1]:
            if (pawn_mat[a,col]==collab1_mat[b,col1]):
                print (feature_names[col], ' - ', pawn_mat[a, col])


In [ ]:
for col in pawn_mat[0].nonzero()[1]:
        for col1 in collab1_mat[0].nonzero()[1]:
            if (pawn_mat[0,col]==collab1_mat[0,col1]):
                print (feature_names[col], ' - ', pawn_mat[0, col])


In [ ]:
print (y2)